In [1]:
import re
import os
import random
import numpy as np
import pandas as pd
import pickle

# Add random features/variables which will give no predictive power to classifiers

In [5]:
features = pd.read_csv("../data/all_training_features.csv")

features

,blue,green,red,bg_change,br_change,bg_z,br_z,bg_pixel,br_pixel,hue,...,green_max_temp5,red_max_temp5,greenblue_max5,redblue_max5,greenblue_max_temp5,redblue_max_temp5,validation,ml_class,date,location
0,0.0834,0.1078,0.1146,-0.047917,-0.025527,-0.762222,-0.334684,-0.127615,-0.157576,0.130342,...,177.0,205.0,168.0,163.0,153.0,147.0,-999.0,0.0,2019-01-09,nairobi
1,0.0879,0.1078,0.1176,-0.006089,0.003928,0.676423,0.525378,-0.101686,-0.144526,0.111672,...,177.0,205.0,192.0,174.0,171.0,187.0,-999.0,0.0,2019-01-09,nairobi
2,0.0814,0.1084,0.1220,-0.012099,-0.020424,0.469696,-0.185670,-0.142255,-0.199607,0.110837,...,177.0,205.0,192.0,174.0,171.0,187.0,-999.0,0.0,2019-01-09,nairobi
3,0.1004,0.1104,0.1302,0.052064,0.035775,2.676508,1.455289,-0.047438,-0.129228,0.055928,...,177.0,205.0,168.0,163.0,153.0,147.0,-999.0,0.0,2019-01-09,nairobi
4,0.1060,0.1142,0.1226,0.039450,0.057133,2.242680,2.078900,-0.037239,-0.072616,0.082329,...,177.0,219.0,192.0,174.0,171.0,187.0,-999.0,0.0,2019-01-09,nairobi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769505,0.1250,0.0997,0.0981,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.656753,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2020-06-25,m1
769506,0.1398,0.1122,0.1026,0.139800,0.139800,0.139800,0.139800,0.139800,0.139800,0.623656,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2020-06-25,m1
769507,0.1790,0.1164,0.0880,0.179000,0.179000,0.179000,0.179000,0.179000,0.179000,0.614652,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2020-06-25,m1
769508,0.1460,0.1120,0.1030,0.146000,0.146000,0.146000,0.146000,0.146000,0.146000,0.631783,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2020-06-25,m1


In [6]:
mu, sigma = 0.5, 0.1

features.insert(5,'gr_change', np.random.normal(mu, sigma, len(features)))

features.insert(8,'gr_z', np.random.lognormal(mu, sigma, len(features)))

features.insert(11,'gr_pixel', np.random.random(len(features)))

features

,blue,green,red,bg_change,br_change,gr_change,bg_z,br_z,gr_z,bg_pixel,...,green_max_temp5,red_max_temp5,greenblue_max5,redblue_max5,greenblue_max_temp5,redblue_max_temp5,validation,ml_class,date,location
0,0.0834,0.1078,0.1146,-0.047917,-0.025527,0.371936,-0.762222,-0.334684,1.464716,-0.127615,...,177.0,205.0,168.0,163.0,153.0,147.0,-999.0,0.0,2019-01-09,nairobi
1,0.0879,0.1078,0.1176,-0.006089,0.003928,0.394598,0.676423,0.525378,1.933391,-0.101686,...,177.0,205.0,192.0,174.0,171.0,187.0,-999.0,0.0,2019-01-09,nairobi
2,0.0814,0.1084,0.1220,-0.012099,-0.020424,0.461080,0.469696,-0.185670,1.545889,-0.142255,...,177.0,205.0,192.0,174.0,171.0,187.0,-999.0,0.0,2019-01-09,nairobi
3,0.1004,0.1104,0.1302,0.052064,0.035775,0.515392,2.676508,1.455289,1.699647,-0.047438,...,177.0,205.0,168.0,163.0,153.0,147.0,-999.0,0.0,2019-01-09,nairobi
4,0.1060,0.1142,0.1226,0.039450,0.057133,0.349319,2.242680,2.078900,1.656258,-0.037239,...,177.0,219.0,192.0,174.0,171.0,187.0,-999.0,0.0,2019-01-09,nairobi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769505,0.1250,0.0997,0.0981,0.125000,0.125000,0.563748,0.125000,0.125000,1.569675,0.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2020-06-25,m1
769506,0.1398,0.1122,0.1026,0.139800,0.139800,0.383903,0.139800,0.139800,1.648313,0.139800,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2020-06-25,m1
769507,0.1790,0.1164,0.0880,0.179000,0.179000,0.400353,0.179000,0.179000,1.673361,0.179000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2020-06-25,m1
769508,0.1460,0.1120,0.1030,0.146000,0.146000,0.456697,0.146000,0.146000,1.610649,0.146000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2020-06-25,m1


### subset only "nairobi" rows

In [13]:
features = features[features["location"]=="nairobi"]

In [14]:
features.to_csv("../data/all_training_features.csv", index=True, header=True)

### Check file size - if > 50MB then remove rows randomly to meet githubs storage requirements 

In [15]:
os.path.getsize("../data/all_training_features.csv")/(1024*1024) # gives size in MB

120.1336145401001

In [18]:
features = features.sample(frac=.40) #randomly removes 40% of rows

In [19]:
features.to_csv("../data/all_training_features.csv", index=True, header=True)
os.path.getsize("../data/all_training_features.csv")/(1024*1024) # now csv is 29.2MB

48.0604133605957